# CSCI 4253 / 5253 - Lab #4 - Patent Problem with Spark DataFrames
<div>
 <h2> CSCI 4283 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>

This [Spark cheatsheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf) is useful as is [this reference on doing joins in Spark dataframe](http://www.learnbymarketing.com/1100/pyspark-joins-by-example/).

The [DataBricks company has one of the better reference manuals for PySpark](https://docs.databricks.com/spark/latest/dataframes-datasets/index.html) -- they show you how to perform numerous common data operations such as joins, aggregation operations following `groupBy` and the like.

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

The following aggregation functions may be useful -- [these can be used to aggregate results of `groupby` operations](https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html#example-aggregations-using-agg-and-countdistinct). More documentation is at the [PySpark SQL Functions manual](https://spark.apache.org/docs/2.3.0/api/python/pyspark.sql.html#module-pyspark.sql.functions). Feel free to use other functions from that library.

In [2]:
# Added length and trim to address missing rows. Added coalesce for join
from pyspark.sql.functions import col, count, countDistinct, length, trim, lit,coalesce

In [3]:
spark = SparkSession \
    .builder \
    .appName("Lab4-Dataframe") \
    .master("local[*]")\
    .getOrCreate()

Read in the citations and patents data and check that the data makes sense. Note that unlike in the RDD solution, the data is automatically inferred to be Integer() types.

In [4]:
citations = spark.read.load('cite75_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [5]:
citations.show(5)

+-------+-------+
| CITING|  CITED|
+-------+-------+
|3858241| 956203|
|3858241|1324234|
|3858241|3398406|
|3858241|3557384|
|3858241|3634889|
+-------+-------+
only showing top 5 rows



In [6]:
patents = spark.read.load('apat63_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [7]:
patents.show(5)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
|3070801| 1963| 1096|   NULL|     BE|   NULL|    NULL|      1|  NULL|   269|  6|    69| NULL|       1|    NULL|    0.0|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|
|3070802| 1963| 1096|   NULL|     US|     TX|    NULL|      1|  NULL|     2|  6|    63| NULL|       0|    NULL|   NULL|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|
|3070803| 1963| 1096|   NULL|     US|     IL|    NULL|      1|  NULL|     2|  6|    6

### Approach
1) Create aliases for the citations DataFrame and two copies for the patents DataFrame where one represents the CITED patent and the other represents the CITING patent. This allows the patent DataFrame to be joined twice so that we can pull off the CITED state and the CITING state.
2) Calculate the co-state citation count
- Join the CITED patent from the citations DataFrame with the patents table on CITED.
- Join the CITING patent from the citations DataFrame with the patents table on CITING.
- Keep records where the cited state is equal to the citing state, so long as neither cited state nor citing state are missing.

3) Group the filtered results by the citing patent using `.groupBy()`.
4) Aggregate the grouped records using `.count()` to calculate the number of co-state citations per citing patent to `CO_CITED_COUNT`.
5) Left join the co-state citation count with the full patent DataFrame. Include those with zero co-state citations. Missing counts are replaced with zero using `coalesce()`.
6) Order by descending co-state count using `orderBy()` and `.desc`.
7) Display the top ten patents using `limit()` and `.show()`.

In [8]:
# aliases
c = citations.alias("c")
p_cited  = patents.alias("p_cited")
p_citing = patents.alias("p_citing")

co_state_counts = (
    c.join(p_cited,  col("c.CITED")  == col("p_cited.PATENT"),  "inner")
     .join(p_citing, col("c.CITING") == col("p_citing.PATENT"), "inner")
     .where(
         (col("p_cited.POSTATE") == col("p_citing.POSTATE")) &
         (length(trim(col("p_cited.POSTATE")))  > 0) &
         (length(trim(col("p_citing.POSTATE"))) > 0)
     )
     .groupBy(col("c.CITING").alias("PATENT"))
     .agg(count(lit(1)).alias("CO_CITED_COUNT"))   # equivalent to COUNT(*)
)

result_df = (
    patents.alias("p")
      .join(co_state_counts.alias("cs"), col("p.PATENT") == col("cs.PATENT"), "left")
      .select(
          "p.*",
          coalesce(col("cs.CO_CITED_COUNT"), lit(0)).alias("CO_CITED_COUNT")
      )
      .orderBy(col("CO_CITED_COUNT").desc())
      .limit(10)
)

result_df.show(truncate=False)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+--------------+
|PATENT |GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|CO_CITED_COUNT|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+--------------+
|5959466|1999 |14515|1997   |US     |CA     |5310    |2      |NULL  |326   |4  |46    |159  |0       |1.0     |NULL   |0.6186  |NULL    |4.8868  |0.0455  |0.044   |NULL    |NULL    |125           |
|5983822|1999 |14564|1998   |US     |TX     |569900  |2      |NULL  |114   |5  |55    |200  |0       |0.995   |NULL   |0.7201  |NULL    |12.45   |0.0     |0.0     |NULL    |NULL    |103           |
|6008204|1